In [1]:
import pandas as pd
import sqlite3

from duckdb.duckdb import query

In [2]:
db_path = '../data/clean/cyclistic.db'


In [3]:
conn = sqlite3.connect(db_path)
print("Connected to database successfully!")

Connected to database successfully!


In [5]:
query0 = """
DELETE FROM cyclistic_trips
WHERE
    ride_id IS NULL OR
    rideable_type IS NULL OR
    started_at IS NULL OR
    ended_at IS NULL OR
    start_station_name IS NULL OR
    start_station_id IS NULL OR
    end_station_name IS NULL OR
    end_station_id IS NULL OR
    start_lat IS NULL OR
    start_lng IS NULL OR
    end_lat IS NULL OR
    end_lng IS NULL OR
    member_casual IS NULL OR
    ride_length_min IS NULL;
"""
cursor = conn.cursor()
cursor.execute(query0)
conn.commit()
print(f"Removed {cursor.rowcount} rows with NULL values")


Removed 0 rows with NULL values


In [7]:
query1 = """
SELECT
    member_casual AS user_type,
    AVG(ride_length_min) AS avg_ride_length_min,
    COUNT(*) AS total_ride_length_min
FROM cyclistic_trips
GROUP BY member_casual
"""

In [8]:
avg_duration_df = pd.read_sql(query1, conn)
print(avg_duration_df)

avg_duration_df.to_csv('../outputs/tables/avg_ride_duration_by_type.csv', index=False)

  user_type  avg_ride_length_min  total_ride_length_min
0    casual            23.953679                1521432
1    member            12.448205                2686516


In [9]:
query2 = ("""
SELECT
    day_of_week,
    member_casual AS user_type,
    COUNT(*) AS number_of_rides
FROM cyclistic_trips
GROUP BY day_of_week, user_type
ORDER BY
    CASE day_of_week
        WHEN 'Monday' THEN 1
        WHEN 'Tuesday' THEN 2
        WHEN 'Wednesday' THEN 3
        WHEN 'Thursday' THEN 4
        WHEN 'Friday' THEN 5
        WHEN 'Saturday' THEN 6
        WHEN 'Sunday' THEN 7
    END""")

In [10]:
rides_by_day_df = pd.read_sql_query(query2, conn)
print(rides_by_day_df)
rides_by_day_df.to_csv('../outputs/tables/rides_by_day_of_week.csv', index=False)
print("Query 2 results saved")

   day_of_week user_type  number_of_rides
0       Monday    casual           181642
1       Monday    member           399232
2      Tuesday    casual           162819
3      Tuesday    member           421399
4    Wednesday    casual           187342
5    Wednesday    member           446866
6     Thursday    casual           183059
7     Thursday    member           414236
8       Friday    casual           222616
9       Friday    member           376718
10    Saturday    casual           317804
11    Saturday    member           332556
12      Sunday    casual           266150
13      Sunday    member           295509
Query 2 results saved


In [11]:
query3 = """
SELECT
    hour,
    member_casual AS user_type,
    COUNT(*) AS number_of_rides
FROM cyclistic_trips
GROUP BY hour, user_type
ORDER BY hour;
"""

In [12]:
rides_by_hour_df = pd.read_sql_query(query3, conn)
print(rides_by_hour_df)
rides_by_hour_df.to_csv('../outputs/tables/rides_by_hour.csv', index=False)
print("Query 3 results saved!")

    hour user_type  number_of_rides
0      0    casual            22283
1      0    member            19664
2      1    casual            14795
3      1    member            11507
4      2    casual             8712
5      2    member             6220
6      3    casual             4737
7      3    member             4248
8      4    casual             3723
9      4    member             5433
10     5    casual             8254
11     5    member            25804
12     6    casual            20089
13     6    member            80488
14     7    casual            38746
15     7    member           156714
16     8    casual            54376
17     8    member           193854
18     9    casual            55208
19     9    member           127251
20    10    casual            70298
21    10    member           111491
22    11    casual            89531
23    11    member           129733
24    12    casual           104717
25    12    member           147855
26    13    casual          

In [13]:
query4 = """
SELECT
    start_station_name,
    COUNT(*) AS number_of_casual_rides
FROM cyclistic_trips
WHERE member_casual = 'casual'
    AND start_station_name IS NOT NULL
GROUP BY start_station_name
ORDER BY number_of_casual_rides DESC
LIMIT 10;
"""

In [14]:
top_casual_stations_df = pd.read_sql_query(query4, conn)
print(top_casual_stations_df)
top_casual_stations_df.to_csv('../outputs/tables/top_10_casual_stations.csv', index=False)
print("Query 4 results saved!")

                   start_station_name  number_of_casual_rides
0             Streeter Dr & Grand Ave                   48314
1   DuSable Lake Shore Dr & Monroe St                   32194
2               Michigan Ave & Oak St                   23294
3  DuSable Lake Shore Dr & North Blvd                   21356
4                     Millennium Park                   20838
5                      Shedd Aquarium                   20117
6                      Dusable Harbor                   17262
7                 Theater on the Lake                   15398
8               Michigan Ave & 8th St                   12554
9                   Adler Planetarium                   12272
Query 4 results saved!


In [15]:
query5 = """
SELECT
    rideable_type,
    member_casual AS user_type,
    COUNT(*) AS number_of_rides
FROM cyclistic_trips
GROUP BY rideable_type, user_type;
"""

In [16]:
bike_type_df = pd.read_sql_query(query5, conn)
print(bike_type_df)
bike_type_df.to_csv('../outputs/tables/bike_type_preference.csv', index=False)
print("Query 5 results saved!")

      rideable_type user_type  number_of_rides
0      classic_bike    casual           968748
1      classic_bike    member          1759173
2     electric_bike    casual           526944
3     electric_bike    member           905256
4  electric_scooter    casual            25740
5  electric_scooter    member            22087
Query 5 results saved!


In [21]:
conn.close()
print("Database connection closed. Analysis complete!")

Database connection closed. Analysis complete!
